In [1]:
import pubmed_parser as pp

dicts_out = pp.parse_medline_xml('data/pubmed_data/raw_data/44_abstracts_test.xml',
                                 year_info_only=False,
                                 nlm_category=False,
                                 author_list=False,
                                 reference_list=False)

In [2]:
dicts_out[0].keys()

dict_keys(['title', 'abstract', 'journal', 'authors', 'pubdate', 'pmid', 'pmc', 'mesh_terms', 'publication_types', 'chemical_list', 'keywords', 'doi', 'references', 'delete', 'affiliations', 'medline_ta', 'nlm_unique_id', 'issn_linking', 'country'])

In [3]:
import os

files = os.listdir("/home/rgoli/MetaMap-src/data/gold_standard/AnnotationResults_Abstracts/done")
print(files)

manual_pmids = [file.split('.')[0][4:] for file in files if file!='.ipynb_checkpoints']
print(manual_pmids)

['ABS_11604796.txt.done', 'ABS5_20841799.txt.done', 'ABS_29295283.txt.done', 'ABS_30306934.txt.done', 'ABS_28361157.txt.done', 'ABS_30871678.txt.done', 'ABS_29076113.txt.done', 'ABS5_22052898.txt.done', 'ABS_29433958.txt.done', 'ABS_19448882.txt.done', 'ABS_14664046.txt.done', 'ABS_24197356.txt.done', 'ABS_20501347.txt.done', 'ABS_29169646.txt.done', 'ABS_23304299.txt.done', 'ABS_29461493.txt.done', 'ABS_26133480.txt.done', 'ABS_18002176.txt.done', 'ABS_11198187.txt.done', 'ABS_18779083.txt.done', 'ABS_11418542.txt.done', 'ABS_26806717.txt.done', 'ABS5_12087117.txt.done', 'ABS_27401856.txt.done', 'ABS5_27301749.txt.done', 'ABS_28676255.txt.done', 'ABS_26539547.txt.done', 'ABS_22269224.txt.done', 'ABS_31438026.txt.done', 'ABS_31438019.txt.done', 'ABS_15360885.txt.done', 'ABS_17238374.txt.done', 'ABS_22326287.txt.done', 'ABS_23232759.txt.done', 'ABS_26590980.txt.done', 'ABS_22019377.txt.done', 'ABS5_23828174.txt.done', 'ABS_25109270.txt.done', 'ABS_26474836.txt.done', 'ABS_23523876.txt.d

In [4]:
# import pandas as pd

# raw_data = pd.DataFrame.from_dict(dicts_out)

# raw_data.head()

In [5]:
# filter_raw_data = raw_data[raw_data['pmid'].isin(manual_pmids)]

In [6]:
# filter_raw_data.head()

In [7]:
pmid_manual_kw={}
for file in files:
    if file=='.ipynb_checkpoints': continue
    with open("/home/rgoli/MetaMap-src/data/gold_standard/AnnotationResults_Abstracts/done/"+file,'r') as fp:
        temp = fp.read().splitlines()
        pmid_manual_kw[temp[0].strip()]=temp[1:]

print(pmid_manual_kw['11604796'])

['drug-ordering system', 'optimizing', 'guideline-based ordering system ', 'enable', 'prescription errors ', 'operates', 'background process ', 'triggered', 'rules ', 'directs', 'browsing', 'development', 'clinical practice guidelines', 'developed', 'disseminate', 'design', 'decision tree']


In [11]:
import re

for paper in dicts_out[11:14]:
    tit_text = paper['title']
    abs_text = paper['abstract']
    
    print(paper['pmid'])
    print(tit_text)
    print('-'*20)
    print(abs_text)
    print('-'*20)
    
    ## Find KW/KP with Acronyms
    definitions = {}
    for acronym in re.findall(r'\(([A-Za-z\-]+?)\)', abs_text):
        length = len(list(filter(str.isupper, acronym)))
        print(acronym, length)
        
        match = re.search(r'(?:\w+\W+){'+str(length)+r'}\('+acronym+r'\)',abs_text)
        
        if match:
            definitions[match.group()]=match.span()
#             print("Loc: Abstract\tPosition: ",match.span(), "\tWord: ", match.group())

#     print(", ".join(definitions))
    print(definitions)
       
    ## Find KW/KP Positions and Locations
    for kw in pmid_manual_kw[paper['pmid']]:
        kw=kw.strip()
        tit_search = re.search(kw,tit_text, re.IGNORECASE)
        abs_search = re.search(kw,abs_text, re.IGNORECASE)
        if tit_search:
            print("Loc: Title\tPosition: ",tit_search.span(), "\tWord: ", tit_search.group())
        elif abs_search:
            print("Loc: Abstract\tPosition: ",abs_search.span(), "\tWord: ", abs_search.group())
        elif kw in definitions.keys():
            print("Loc: Abstract\tPosition: ",definitions[kw], "\tWord: ", kw)
        else:
            print(kw,' NOT found!!!')
            
    print("*"*40)

31438026
An Ontology-Based Personalized Decision Support System for Use in the Complex Chronically Ill Patient.
--------------------
Management of the complex chronically ill patient is highly problematic. This is due to the need to complement recommendations in clinical guidelines with recommendations based on treatments performed on a representative set of patients. It is necessary to identify conflicts between the recommendations of different guidelines designed for handling specific chronic diseases. The PITeS-TIiSS project (code PI15/01213) aims to overcome this problem by designing and deploying an ontology-based personalized clinical decision support tool. This helps to improve personalized decisions and reduces variability in clinical practice in an integrated care environment. This paper describes the methodology followed in developing the ontology used to infer clinical knowledge and to develop and implement the support tool. The tool will access the information provided by t

In [12]:
import re
# from itertools import imap

for paper in dicts_out[11:14]:
    tit_text = paper['title']
    abs_text = paper['abstract']
    
    print(paper['pmid'])
    print(tit_text)
    print('-'*20)
    print(abs_text)
    print('-'*20)
    
#     imap(abs_text.__contains__, pmid_manual_kw[paper['pmid']])
    
    for kw in pmid_manual_kw[paper['pmid']]:
        kw=kw.strip()
        if kw in tit_text:
            print("Loc: Title\tPosition: ",tit_text.find(kw),"\tWord: ",kw)
            print([(m.start(0), m.end(0)) for m in re.finditer(kw,tit_text,re.IGNORECASE)])
        elif kw in abs_text:
            print("Loc: Title\tPosition: ",abs_text.find(kw),"\tWord: ",kw)
            print([(m.start(0), m.end(0)) for m in re.finditer(kw,abs_text,re.IGNORECASE)])
        else:
            print(kw," NOT found!!!")
    print("*"*40)

31438026
An Ontology-Based Personalized Decision Support System for Use in the Complex Chronically Ill Patient.
--------------------
Management of the complex chronically ill patient is highly problematic. This is due to the need to complement recommendations in clinical guidelines with recommendations based on treatments performed on a representative set of patients. It is necessary to identify conflicts between the recommendations of different guidelines designed for handling specific chronic diseases. The PITeS-TIiSS project (code PI15/01213) aims to overcome this problem by designing and deploying an ontology-based personalized clinical decision support tool. This helps to improve personalized decisions and reduces variability in clinical practice in an integrated care environment. This paper describes the methodology followed in developing the ontology used to infer clinical knowledge and to develop and implement the support tool. The tool will access the information provided by t

In [14]:
results = []
for paper in dicts_out:
    tit_text = paper['title'].lower()
    abs_text = paper['abstract'].lower()
    pmid = paper['pmid']
    print(pmid)
    print(tit_text)
    print('-'*100)
    print(abs_text)
    print('-'*100)
    
#     imap(abs_text.__contains__, pmid_manual_kw[paper['pmid']])
    total_kw = [x.strip().lower() for x in pmid_manual_kw[paper['pmid']]]
    
    position=[]
    for kw in total_kw:
        if kw in tit_text:
            first_occurance = tit_text.find(kw)
            last_position = position
            position = [(m.start(0), m.end(0)) for m in re.finditer(kw,tit_text,re.IGNORECASE)]
            if len(position)>1 and len(last_position)==1:
                for cpos in position:
                    if cpos[0] > last_position[0][1]:
                        position = [cpos]
            elif len(position)==0:
                position =[(first_occurance, first_occurance+len(kw))]
            print("Loc: Title\tFirstOccurance: ",first_occurance,'\tPosition: ',position,"\tWord: ",kw)
            results.append([pmid,'T',first_occurance,position,kw])
        elif kw in abs_text:
            first_occurance = abs_text.find(kw)
            last_position = position
            position = [(m.start(0), m.end(0)) for m in re.finditer(kw,abs_text,re.IGNORECASE)]
            if len(position)>1 and len(last_position)==1:
                for cpos in position:
                    if cpos[0] > last_position[0][1]:
                        position = [cpos]
            elif len(position)==0:
                position =[(first_occurance, first_occurance+len(kw))]
            print("Loc: Abstract\tFirstOccurance: ",first_occurance,'\tPosition: ',position,"\tWord: ",kw)
            results.append([pmid,'A',first_occurance,position,kw])
        else:
            print(kw," NOT found!!!")
    print("*"*100)

11604796
asti: a guideline-based drug-ordering system for primary care.
----------------------------------------------------------------------------------------------------
existing computer-based ordering systems for physicians provide effective drug-centered checks but offer little assistance for optimizing the overall patient-centered treatment strategy. evidence-based clinical practice guidelines have been developed to disseminate state-of-the-art information concerning treatment strategy but these guidelines are poorly used in routine practice. the asti project aims to design a guideline-based ordering system to enable general practitioners to avoid prescription errors and to improve compliance with best therapeutic practices. the " critic mode " operates as a background process and corrects the physician's prescription on the basis of automatically triggered elementary rules that account for isolated guideline recommendations. the " guided mode " directs the physician to the best

In [144]:
len(results)

1109

In [15]:
results = []
for paper in dicts_out:
    tit_text = paper['title'].lower()
    abs_text = paper['abstract'].lower()
    pmid = paper['pmid']
    print(pmid)
    print(tit_text)
    print('-'*100)
    print(abs_text)
    print('-'*100)
    
#     imap(abs_text.__contains__, pmid_manual_kw[paper['pmid']])
    total_kw = [x.strip().lower() for x in pmid_manual_kw[paper['pmid']]]
    
    position=[]
    for kw in total_kw:
        if kw in tit_text:
            first_occurance = tit_text.find(kw)
            last_position = position
            position = [(m.start(0), m.end(0)) for m in re.finditer(kw,tit_text,re.IGNORECASE)]
            if len(position)>1 and len(last_position)==1:
                for cpos in position:
                    if cpos[0] > last_position[0][1]:
                        position = [cpos]
                        break
                    else:
                        position = [cpos]
            elif len(position)==0:
                position =[(first_occurance, first_occurance+len(kw))]
            print("Loc: Title\tFirstOccurance: ",first_occurance,'\tPosition: ',position,"\tWord: ",kw)
            results.append([pmid,'T',first_occurance,position,kw])
        elif kw in abs_text:
            first_occurance = abs_text.find(kw)
            last_position = position
            position = [(m.start(0), m.end(0)) for m in re.finditer(kw,abs_text,re.IGNORECASE)]
            if len(position)>1 and len(last_position)==1:
                for cpos in position:
                    if cpos[0] > last_position[0][1]:
                        position = [cpos]
                        break
                    else:
                        position = [cpos]
            elif len(position)==0:
                position =[(first_occurance, first_occurance+len(kw))]
            print("Loc: Abstract\tFirstOccurance: ",first_occurance,'\tPosition: ',position,"\tWord: ",kw)
            results.append([pmid,'A',first_occurance,position,kw])
        else:
            print(kw," NOT found!!!")
    print("*"*100)

11604796
asti: a guideline-based drug-ordering system for primary care.
----------------------------------------------------------------------------------------------------
existing computer-based ordering systems for physicians provide effective drug-centered checks but offer little assistance for optimizing the overall patient-centered treatment strategy. evidence-based clinical practice guidelines have been developed to disseminate state-of-the-art information concerning treatment strategy but these guidelines are poorly used in routine practice. the asti project aims to design a guideline-based ordering system to enable general practitioners to avoid prescription errors and to improve compliance with best therapeutic practices. the " critic mode " operates as a background process and corrects the physician's prescription on the basis of automatically triggered elementary rules that account for isolated guideline recommendations. the " guided mode " directs the physician to the best